In [ ]:
from pymatgen.io.vasp import Vasprun
from pymatgen.electronic_structure.plotter import DosPlotter

v = Vasprun("/scratch/ba3g18/VASP/Testing/Ads/Li-F_ontop_solvent/ontop_1.50/vasprun.xml")  # Parse vasprun.xml
tdos = v.tdos  # Get total DOS

plotter = DosPlotter()
plotter.add_dos("Total DOS", tdos)
plotter.show(xlim=(-10, 10) ) 

In [1]:
from ase.io import read
import numpy as np

atoms = read("/scratch/ba3g18/VASP/Lithium/Production/Solvated/test/slab_calc/vasprun.xml")
print(np.max(atoms.get_forces()))


0.01222959


In [1]:
from ase.io import read
import numpy as np


def convert_capacitance(c_vasp, surface_area_ang2):
    """
    Convert capacitance from VASP units (e/V) to standard electrochemical units (μF/cm²)
    
    Args:
        c_vasp: Capacitance in e/V (electrons per volt) from VASP
        surface_area_ang2: Surface area in square Angstroms
    
    Returns:
        c_standard: Capacitance in μF/cm² (microfarads per square centimeter)
    """
    # Constants
    elementary_charge = 1.602176634e-19  # C
    ang2_to_cm2 = 1e-16  # 1 Å² = 1e-16 cm²
    C_to_microF = 1e6  # 1 Coulomb/Volt = 1 Farad = 1e6 μF
    
    # Surface area in cm²
    surface_area_cm2 = surface_area_ang2 * ang2_to_cm2
    
    # Convert e/V to C/V first
    c_farad = c_vasp * elementary_charge
    
    # Convert to μF/cm²
    c_standard = (c_farad * C_to_microF) / surface_area_cm2
    
    return c_standard


def convert_capacitance_reverse(c_standard, surface_area_ang2):
    """
    Convert capacitance from standard electrochemical units (μF/cm²) to VASP units (e/V)
    
    Args:
        c_standard: Capacitance in μF/cm² (microfarads per square centimeter)
        surface_area_ang2: Surface area in square Angstroms
    
    Returns:
        c_vasp: Capacitance in e/V (electrons per volt) for VASP
    """
    # Constants
    elementary_charge = 1.602176634e-19  # C
    ang2_to_cm2 = 1e-16  # 1 Å² = 1e-16 cm²
    microF_to_C = 1e-6  # 1 μF = 1e-6 F = 1e-6 C/V
    
    # Surface area in cm²
    surface_area_cm2 = surface_area_ang2 * ang2_to_cm2
    
    # Convert μF/cm² to C/V (Farads)
    c_farad = c_standard * surface_area_cm2 * microF_to_C
    
    # Convert C/V to e/V
    c_vasp = c_farad / elementary_charge
    
    return c_vasp

atoms = read("/scratch/ba3g18/VASP/Testing/Solvation/ase_surface/PZC/slab_calc/vasprun.xml")
area =  np.linalg.norm(np.cross(atoms.cell[0], atoms.cell[1])) 
c_vasp =   0.10220302
 

c_standard = convert_capacitance(c_vasp, area)
print(f"Capacitance in standard units: {c_standard:.4f} μF/cm²")



Capacitance in standard units: 13.8375 μF/cm²


In [ ]:
from ase.io import read

# ase energy 
atoms = read("/scratch/ba3g18/VASP/Testing/Ads/Li-F_ontop_solvent/ontop_1.80/vasprun.xml")
print(atoms.get_potential_energy())

# pymatgen energy
from pymatgen.io.vasp.outputs import Vasprun

v = Vasprun("/scratch/ba3g18/VASP/Testing/Ads/Li-F_ontop_solvent/ontop_1.80/vasprun.xml")
# Get the energy of the system
energy = v.final_energy
print("Energy:", energy)

In [ ]:

from pymatgen.io.vasp.outputs import Vasprun

v = Vasprun("/scratch/ba3g18/VASP/Testing/Ads/Li-F_ontop_solvent/ontop_1.80/vasprun.xml")
v.get_computed_entry()

In [11]:

from pymatgen.io.vasp.outputs import Vasprun

v = Vasprun("/scratch/ba3g18/VASP/Testing/Reference/osani/Li_slab_efermi_-1.00/vasprun.xml")
v.calculate_efermi()

ParseError: no element found: line 1511, column 0 (<string>)

In [ ]:
from pymatgen.core.structure import Structure
import nglview

structure = Structure.from_file("/scratch/ba3g18/VASP/Testing/Ads/Li-F_ontop_solvent/ontop_1.60/POSCAR")
view = nglview.show_pymatgen(structure)
view

In [ ]:
import os
import matplotlib.pyplot as plt
from pymatgen.io.vasp import Outcar

# 1. Specify the path to your VASP directory
directory = "/scratch/ba3g18/VASP/Testing/Ads/Li-F_ontop_solvent/ontop_1.60"

# 2. Read OUTCAR
try:
    outcar = Outcar(os.path.join(directory, "OUTCAR"))
except FileNotFoundError:
    print("OUTCAR not found. Plotting skipped.")
    exit()

# 3. Extract Data
iterations = range(1, len(outcar.ionic_steps) + 1)
energies = [step["e_0_energy"] for step in outcar.ionic_steps]
max_forces = [max(np.linalg.norm(step["forces"], axis=1)) for step in outcar.ionic_steps] # get the maximum force per iteration.
import numpy as np

# 4. Plot Energy vs. Iteration
plt.figure()
plt.plot(iterations, energies, marker='o')
plt.xlabel("Iteration")
plt.ylabel("Energy (eV)")
plt.title("Energy vs. Iteration")
plt.grid(True)
plt.show()

# 5. Plot Max Force vs. Iteration
plt.figure()
plt.plot(iterations, max_forces, marker='o')
plt.xlabel("Iteration")
plt.ylabel("Max Force (eV/Å)")
plt.title("Max Force vs. Iteration")
plt.grid(True)
plt.show()

In [ ]:
from pymatgen.io.vasp import Vasprun
from pymatgen.electronic_structure.plotter import DosPlotter

try:
    v = Vasprun("/scratch/ba3g18/VASP/Testing/Ads/Li-F_ontop_solvent/ontop_1.50/vasprun.xml")
    dos = v.complete_dos  # Get the complete DOS object
    plotter = DosPlotter()
    # plotter.add_dos_dict(dos.get_element_spd_dos())  # Plot element-resolved PDOS
    plotter.add_dos_dict(dos.get_site_spd_dos()) # Plot site-resolved PDOS
    plotter.show()
except FileNotFoundError:
    print("vasprun.xml not found. PDOS plotting skipped.")

In [ ]:
from pymatgen.io.vasp import Locpot
import matplotlib.pyplot as plt
import numpy as np

# 1. Parse the LOCPOT file
locpot = Locpot.from_file("/scratch/ba3g18/VASP/Testing/Ads/Li-F_ontop_solvent/ontop_1.50/LOCPOT")

# 2. Extract the potential data
potential_data = locpot.data["total"]

# 3. Calculate the XY-averaged potential
xy_averaged_potential = np.mean(potential_data, axis=(0, 1))

# 4. Get the Z-coordinates
z_coords = np.linspace(0, locpot.structure.lattice.c, len(xy_averaged_potential))

# 5. Plot the XY-averaged potential along Z
plt.plot(z_coords, xy_averaged_potential)
plt.xlabel("Z (Angstrom)")
plt.ylabel("XY-averaged Potential (V)")
plt.title("XY-averaged Electrostatic Potential along Z")
plt.show()

In [ ]:
from pymatgen.io.vasp import Vasprun
from pymatgen.electronic_structure.plotter import BSPlotter

v = Vasprun("/scratch/ba3g18/VASP/Testing/Ads/Li-F_ontop_solvent/ontop_1.50/vasprun.xml")
bs = v.get_band_structure()
plotter = BSPlotter(bs)
plotter.show()

In [ ]:
from pymatgen.io.vasp import Chgcar
import matplotlib.pyplot as plt

chgcar = Chgcar.from_file("CHGCAR")
# Example, plot a slice of the charge density.
plt.imshow(chgcar.data['total'][:,:,chgcar.data['total'].shape[2]//2])
plt.show()

In [ ]:
from pymatgen.io.vasp import Locpot
import matplotlib.pyplot as plt
import numpy as np

# Load LOCPOT and PHISOLV
locpot = Locpot.from_file("/scratch/ba3g18/VASP/Testing/Ads/Li-F_ontop_solvent/ontop_1.55/LOCPOT")
phisolv = Locpot.from_file("/scratch/ba3g18/VASP/Testing/Ads/Li-F_ontop_solvent/ontop_1.55/PHISOLV") # PHISOLV is also a LOCPOT-like file

# Extract potential data
total_potential = locpot.data["total"]
solvent_potential = phisolv.data["total"]

# XY-averaged potentials
total_potential_avg = np.mean(total_potential, axis=(0, 1))
solvent_potential_avg = np.mean(solvent_potential, axis=(0, 1))

# Z-coordinates
z_coords = np.linspace(0, locpot.structure.lattice.c, len(total_potential_avg))

# Plotting
plt.plot(z_coords, total_potential_avg, label="Total Potential")
plt.plot(z_coords, solvent_potential_avg, label="Solvent Potential")
plt.xlabel("Z (Angstrom)")
plt.ylabel("XY-averaged Potential (V)")
plt.title("Electrostatic Potential Profiles")
plt.legend()
plt.show()

# Calculate the difference to see the solvent effect
potential_difference = total_potential_avg - solvent_potential_avg
plt.plot(z_coords, potential_difference, label="Potential Difference", color='green')
plt.xlabel("Z (Angstrom)")
plt.ylabel("Potential Difference (V)")
plt.title("Solvent Effect on Potential")
plt.legend()
plt.show()

In [ ]:
import os
from pymatgen.io.vasp import Locpot, Outcar
import matplotlib.pyplot as plt
import numpy as np

# --- 1. Specify the Path ---
directory = "/scratch/ba3g18/VASP/Testing/Ads/Li-F_ontop_solvent/ontop_1.55"

# --- 2. Construct File Paths ---
locpot_path = os.path.join(directory, "LOCPOT")
phisolv_path = os.path.join(directory, "PHISOLV")
vsolv_path = os.path.join(directory, "VSOLV")
rhob_path = os.path.join(directory, "RHOB")
rhoion_path = os.path.join(directory, "RHOION")
outcar_path = os.path.join(directory, "OUTCAR")

# --- 3. Load Data ---
try:
    locpot = Locpot.from_file(locpot_path)
    phisolv = Locpot.from_file(phisolv_path)
    vsolv = Locpot.from_file(vsolv_path)
    rhob = Locpot.from_file(rhob_path)
    rhoion = Locpot.from_file(rhoion_path)
    outcar = Outcar(outcar_path)

except FileNotFoundError as e:
    print(f"Error: Required file not found: {e}")
    exit()

# --- 4. Extract Data ---
total_potential = locpot.data["total"]
solvent_potential = phisolv.data["total"]
vsolv_potential = vsolv.data["total"]
bound_charge = rhob.data["total"]
ionic_charge = rhoion.data["total"]

# XY-averaged profiles
total_potential_avg = np.mean(total_potential, axis=(0, 1))
solvent_potential_avg = np.mean(solvent_potential, axis=(0, 1))
vsolv_potential_avg = np.mean(vsolv_potential, axis=(0, 1))
bound_charge_avg = np.mean(bound_charge, axis=(0, 1))
ionic_charge_avg = np.mean(ionic_charge, axis=(0, 1))

# Z-coordinates
z_coords = np.linspace(0, locpot.structure.lattice.c, len(total_potential_avg))

# Solvation Energy
final_energy = outcar.final_energy

# --- 5. Plotting ---

# --- 5.1. Electrostatic Potentials ---
plt.figure(figsize=(10, 6))
plt.plot(z_coords, total_potential_avg, label="Total Potential (LOCPOT)")
plt.plot(z_coords, solvent_potential_avg, label="Solvent Potential (PHI_SOLV)")
plt.plot(z_coords, vsolv_potential_avg, label="Cavity Correction (VSOLV)")
plt.xlabel("Z (Angstrom)")
plt.ylabel("XY-averaged Potential (V)")
plt.title("Electrostatic Potentials along Z")
plt.legend()
plt.grid(True)
plt.show()

# --- 5.2. Charge Densities ---
plt.figure(figsize=(10, 6))
plt.plot(z_coords, bound_charge_avg, label="Bound Charge (RHOB)")
plt.plot(z_coords, ionic_charge_avg, label="Ionic Charge (RHOION)")
plt.xlabel("Z (Angstrom)")
plt.ylabel("XY-averaged Charge Density (e/Angstrom^3)")
plt.title("Charge Densities along Z")
plt.legend()
plt.grid(True)
plt.show()

# --- 5.3. Solvation Energy ---
print(f"Final Energy (with solvation): {final_energy} eV")
# To get the solvation energy you would need to run the calculation without
# solvation, and then subtract the energies.

In [3]:
from ase.io import read
from ase.visualize import view
import numpy as np
from scipy.spatial import ConvexHull
import urllib.request
import os

# Download EC structure from PubChem (CID: 7303)
if not os.path.exists('ec.sdf'):
    urllib.request.urlretrieve('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/CID/7303/record/SDF/?record_type=3d', 'ec.sdf')

# Read the structure
ec = read('ec.sdf')

# Calculate radii
positions = ec.get_positions()
center_of_mass = ec.get_center_of_mass()
distances = np.linalg.norm(positions - center_of_mass, axis=1)
max_radius = np.max(distances)
radius_of_gyration = np.sqrt(np.mean(distances**2))

# Maximum pairwise distance / 2
max_distance = 0
for i in range(len(ec)):
    for j in range(i+1, len(ec)):
        distance = np.linalg.norm(positions[i] - positions[j])
        if distance > max_distance:
            max_distance = distance
radius_from_diameter = max_distance / 2

# Volume-based approach
hull = ConvexHull(positions)
volume = hull.volume
radius_from_volume = (3 * volume / (4 * np.pi))**(1/3)

print(f"EC Radius Estimates:")
print(f"Maximum radius from center of mass: {max_radius:.2f} Å")
print(f"Radius from maximum diameter: {radius_from_diameter:.2f} Å")
print(f"Radius of gyration: {radius_of_gyration:.2f} Å")
print(f"Radius from volume: {radius_from_volume:.2f} Å")
print(f"Average of all methods: {np.mean([max_radius, radius_from_diameter, radius_of_gyration, radius_from_volume]):.2f} Å")

EC Radius Estimates:
Maximum radius from center of mass: 2.48 Å
Radius from maximum diameter: 2.14 Å
Radius of gyration: 1.83 Å
Radius from volume: 1.17 Å
Average of all methods: 1.91 Å


In [1]:
n = 0.8
nmax = 1 / n**3

print(f"nmax: {nmax:.4f} Å")

nmax: 1.9531 Å


In [2]:
# cube root :
import numpy as np
v = 164.0 
cube_root = v**(1/3)
print(f"Cube root of {v}: {cube_root:.4f} bohr")


Cube root of 164.0: 5.4737 bohr
